# 20 뉴스그룹 분류

## 텍스트 데이터 확인
- fetch_20newsgroups: 20개의 뉴스 그룹에서 가져온 데이터
- 20개의 주제에 대한 18846개의 뉴스 데이터

In [2]:
from sklearn.datasets import fetch_20newsgroups
import numpy as np



In [4]:
print(type(news_group))
print(np.array(news_group.data).shape)

NameError: name 'news_group' is not defined

In [ ]:
import pandas as pd

print('target 클래스의 값과 분보도\n',
      pd.Series(news_group.target).value_counts().sort_index()
      )
print('target 클래스의 이름들', news_group.target_names
      )

In [4]:
news_data = fetch_20newsgroups(subset='all', remove=('headers','footers','quotes'),random_state=10)
print(news_data.data[0])

KeyboardInterrupt: 

## 학습/ 평가 데이터 분리

In [3]:
train_news = fetch_20newsgroups(subset='train',remove=('headers','footers','quotes'),random_state=10)
X_train = train_news.data
y_train = train_news.target

train_news = fetch_20newsgroups(subset='test',remove=('headers','footers','quotes'),random_state=10)
X_test = train_news.data
y_test = train_news.target

print(f'학습 데이터 크기:{len(X_train):,}, 평가 데이터크기:{len(X_test):,}')

학습 데이터 크기:11,314, 평가 데이터크기:7,532


## 피처 벡터화 변환/ 모델 생성/ 예측 /평가

### CounterVectorizer 이용


In [4]:
from sklearn.feature_extraction.text import CountVectorizer

cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

# 학습 데이터로 fit()된 CounterVectorizer를 이용하여 테스트 데이터를 featrue extraction 변환
X_train_cnt_vect = cnt_vect.transform(X_test)

print(f'학습데이터 텍스트의 CountVectorizer shape:{X_train_cnt_vect.shape}')

학습데이터 텍스트의 CountVectorizer shape:(7532, 101631)


In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit()

y_hat = lr_clf.predict(X_train_cnt_vect)
print(f'정확도: {accuracy_score(y_test,y_hat):.3f}')

TypeError: LogisticRegression.fit() missing 2 required positional arguments: 'X' and 'y'

### TfidfVectorizer 이용

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(stop_words='english',ngram_range=(1,2),max_df=300)
X_train_tfidf_vect = tfidf_vect.fit_transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression(solver="liblinear")
lr_clf.fit(X_train_tfidf_vect,y_train)

y_hat = lr_clf.predict(X_train_tfidf_vect)
print(f'정확도: {accuracy_score(y_test,y_hat):.3f}')

ValueError: Found input variables with inconsistent numbers of samples: [7532, 11314]

## TfidfVectorizer 
- 불용어 제거 , N-gram (1,2) 적용

## 하이퍼 파라미터 튜닝

In [12]:
from sklearn.model_selection import GridSearchCV

params = {'C':[0.01,0.1,1,5,10]}
grid_cv_lr = GridSearchCV(lr_clf,param_grid=params,cv =3 ,scoring='accuracy')
grid_cv_lr.fit(X_train_tfidf_vect,y_train)
print('최적 파라미터:', grid_cv_lr.best_params_)

y_hat = grid_cv_lr.predict(X_train_tfidf_vect)
# print(f'정확도:{accuracy_score(y_test,y_hat):.3f}')

최적 파라미터: {'C': 10}


ValueError: Found input variables with inconsistent numbers of samples: [7532, 11314]